# 2.Design principles governing the rate of gene expression

## Concept
- Protein degradation rates determine response times for simple gene expression systerms.

## Design principles
- Negative autoregulation accelerates turn-on times but not turn-off times.

## Techniques
- Steady state normalization allows analysis of reponse times.
- Numerical solutions of ODEs using python.
- Using Bokeh plotting.

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade biocircuits watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://biocircuits.github.io/chapters/data/"
else:
    data_path = "data/"
# ------------------------------

import numpy as np
import pandas as pd
import scipy.integrate

import colorcet

import biocircuits.jsplots

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

## Protein stability determines the response time to a change in gene expression

Biological circuits change dynamically even in the constant conditions.

$$
    \frac{dx}{dt}=\beta-\gamma x
$$

If $x_0=0$, then we get:
$$
    x(t)=\frac{\beta}{\gamma}(1-e^{-\gamma t})
$$
Set the $\beta=100$ and $\gamma=1$, we can plot that:


In [2]:
# parameters
beta=100
gamma=1

#profiles
t=np.linspace(0,6,400)
x=beta/gamma*(1-np.exp(-gamma*t))

#plot
p=bokeh.plotting.figure(
    frame_height=275,
    frame_width=375,
    x_axis_label="time",
    y_axis_label="x(t)",
    x_range=[0,6],
)

p.line(t,x,line_width=2)
# Mark the response time (when we get to level 1-1/e)
t0 = 1 / gamma
x0 = beta / gamma * (1 - np.exp(-1))

# Add the glyph with label
source = bokeh.models.ColumnDataSource(
    data=dict(t0=[t0], x0=[x0], text=["response time = 1/γ"])
)
p.circle(x="t0", y="x0", source=source, size=10)
p.add_layout(
    bokeh.models.LabelSet(
        x="t0", y="x0", text="text", source=source, x_offset=10, y_offset=-10
    )
)
p.add_layout(
    bokeh.models.Span(
        location=t0,
        level="underlay",
        dimension="height",
        line_color="black",
        line_dash="dashed",
    )
)
bokeh.io.show(p)

## How can we speed up responses?


In [3]:
# Parameters
beta_1 = 100
gamma = np.array([1, 2, 3])

# Compute dynamics
t = np.linspace(0, 6, 400)
x = [beta_1 / g * (1 - np.exp(-g * t)) for g in gamma]

# Set up plots
colors = bokeh.palettes.Blues5
p1 = bokeh.plotting.figure(
    frame_height=175,
    frame_width=300,
    x_axis_label="time",
    y_axis_label="x(t)",
    x_range=[0, 6],
)
p2 = bokeh.plotting.figure(
    frame_height=175,
    frame_width=300,
    x_axis_label="time",
    y_axis_label="x(t)/xₛₛ",
    x_range=[0, 6],
)
p2.x_range = p1.x_range

# Populate graphs
for x_vals, g, color in zip(x, gamma, colors):
    p1.line(t, x_vals, color=color, line_width=2)
    p1.circle(1 / g, beta_1 / g * (1 - np.exp(-1)), color=color, size=10)
    p2.line(t, x_vals / x_vals.max(), color=color, line_width=2)
    p2.circle(1 / g, 1 - np.exp(-1), color=color, size=10)


# Label lines
p1.text(
    x=[4],
    y=[95],
    text=[f"γ = {gamma[0]}"],
    text_color=colors[0],
    text_font_size="10pt",
    text_align="left",
    text_baseline="top",
)
p1.text(
    x=[4],
    y=[53],
    text=[f"γ = {gamma[1]}"],
    text_color=colors[1],
    text_font_size="10pt",
    text_align="left",
    text_baseline="bottom",
)
p1.text(
    x=[4],
    y=[30],
    text=[f"γ = {gamma[2]}"],
    text_color=colors[2],
    text_font_size="10pt",
    text_align="left",
    text_baseline="top",
)

bokeh.io.show(bokeh.layouts.column([p1, bokeh.models.Spacer(height=20), p2]))

One way to speed up the response time is to increase $\gamma$. However, we also want the steady state to be constant, so we increase $\beta$ by the same factor.

It means that we should increase the speed of producing and the parameter of the speed of decreasing by a same factor the make sure that the response time could be shorter and the steady state could be constant.

In a word, you may find that destabilize a protein can speed its response time.

## Network motifs identify functionally important circuits

Most bacterial proteins and transcription factors are stable, so we cannot use destabilizing to accelerate the response time. Fortunately, we can utilize network motifs to identify circuits to do it.

We define a network motif as a regulatory pattern or sub-circuit that is statistically over-represented in natural networks(circuits), compared to what one might expect from random networks with similar numbers of genes and regulatory interactions.

Network motif analysis could reveal recurring circuit modules with specific functions.

## Negative autoregulation accelerates response times

Using a Hill function:
$$
    \frac{dx}{dt}=\frac{\beta}{1+(x/k)^n}-\gamma x
$$

For arbitrary n, there is no solution for this equation. We would like to talk about the extreme cases.

We assume that $\beta/\gamma k>>1$. 

$$
    x=\begin{cases}
    \beta t,t<k/\beta\\
    k,t\geq k/\beta\\
    \end{cases}
$$

In [4]:
# Curve
t = [0, 1, 2]
x = [0, 1, 1]

# Set up plot
p = bokeh.plotting.figure(
    frame_height=175,
    frame_width=300,
    x_axis_label="time",
    y_axis_label="x(t)",
    x_range=[0, 2],
)

# Custom axis labels
p.xaxis.ticker = bokeh.models.tickers.FixedTicker(ticks=[0, 1])
p.yaxis.ticker = bokeh.models.tickers.FixedTicker(ticks=[0, 1])
p.xaxis.major_label_overrides = {1: "$$k/\\beta$$"}
p.yaxis.major_label_overrides = {1: "$$k$$"}

# Populate plot
p.line(t, x, line_width=2)

# Label slope
p.line([0.2, 0.2, 0.3], [0.2, 0.3, 0.3], color='black')
p.text(
    x=[0.125],
    y=[0.25],
    text=["β"],
    text_color="black",
    text_font_size="12pt",
)

bokeh.io.show(p)

## Highly ultrasensitive, strong repression
For the case

We found that the $1/2$ time of the autorepressed and unregulated situation can be listed:
$$
    \frac{autorepressed t_{1/2}}{unregulated t_{1/2}}\approx \frac{\gamma k}{\beta 2ln2}<<1 
$$

You can see that we have indeed accelerated the response times.


## Less ultrasensitive strong autorepression
